In [ ]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from collections import defaultdict
import os
import pickle
import scipy.stats as st

from itertools import combinations
from scipy.stats import chi2

from code.algorithm_alpha import algorithm_alpha
from code.algorithm_t import algorithm_t

sim = False

In [ ]:
df = pd.read_csv('data/df_level2.csv')
df

In [ ]:
df['STRATIO_scaled'] = preprocessing.scale(df.STRATIO)
df['SCHSIZE_scaled'] = preprocessing.scale(df.SCHSIZE)
df['mean_ESCS_std_scaled'] = preprocessing.scale(df.mean_ESCS_std)

In [ ]:
#df.CNT.unique()
df.groupby('CNT').count().reset_index()['CNT']
#df['Y_MATH'].hist()

In [ ]:
ran_var = False
ran_int = True
n_fix = 1
tol = 0.01  # set as 0.01, 0.05 or 0.10 if algorithm_alpha is used; 
            # otherwise use algorithm_t and tune the tolerance tol
model = 'B' # 'B' for Bernoulli and 'P' for Poisson

variab = 'SE'

In [ ]:
N = df.CNT.nunique()

lengths = np.array(df.groupby('CNT').count().reset_index().iloc[:,1])

if model == 'B':
    y = list(df.groupby('CNT')['Y_BIN_MATH'].apply(np.array).values)
elif model == 'P':
    y = list(df.groupby('CNT')['Y_MATH'].apply(np.array).values)

fix = defaultdict(list)

if variab == "SE":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
    fix[1] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E
elif variab == "S":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
elif variab == "E":
    fix[0] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E

In [ ]:
# with algorithm_alpha

knots, par, W, hess_ran, hess_fix = algorithm_alpha(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N)

In [ ]:
# with algorithm_t

# knots, par, W, hess_ran, hess_fix = algorithm_t(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N)

In [ ]:
group = np.array([np.nan if np.sum(W[i,:])==0 else np.argmax(W[i,:]) for i in range(N)])

df.groupby('CNT').count().reset_index()['CNT']

In [ ]:
def loglikelihood(c, par, group, model):
        s = []  # s <- rep(0,N)
        # param_fixed = par
        # param_random = knots = c

        for i in range(N):

            if group[i]==group[i]:
              a = y[i]

              b = c[int(group[i])]
              for k in range(n_fix):
                b = b + par[k] * fix[k][i]

              if model=='B':
                s.append(np.sum(a * b - np.log(1 + np.exp(b.astype(float) ) ) )) # HO CAMBIATO QUI: ho messo .astype(float)
              elif model=='P':
                s.append(np.sum(a * b - np.exp(b.astype(float)) - np.log(np.nan_to_num(factorial(a))) ))

        return np.sum(np.array(s))

a = loglikelihood(knots, par, group, model)

In [ ]:
file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + str(variab) + '_' + str(tol) + '_' + str(model) + '_merging' + '.pickle'
f = open('output/case_study_results' + file_name, 'wb')
#pickle.dump([par, knots, group, a, W, hess_inv_ran, hess_inv_fix],f)
pickle.dump([knots, par, W, hess_ran, hess_fix, group, a],f)
f.close()

In [ ]:
CI_length = np.array(st.norm.ppf(1 - tol / 2) * np.sqrt(1 / hess_ran))
pd.DataFrame({'lb': knots - CI_length.flatten(), 'ub': knots + CI_length.flatten()})